# Duality properties and sums-of-squares

## A simple problem with equality and inequality constraints

We consider a problem with both equality and inequality constraints

\begin{array}{ll}
\text{minimize}   & x + z\\
\text{subject to} & x - z \geq 0\\
                  & x^2 + z^2 = 1.
\end{array}

In [163]:
include("..\\src\\Polyopt.jl")

Main.Polyopt

In [164]:
x, z = Polyopt.variables(["x", "z"]);
f = x + z;
g = x - z;
h = x^2 + z^2 - 1;
prob = Polyopt.momentprob(2, f, [g], [h]);

In [165]:
X, Z, t, y, solsta = Polyopt.solve_mosek(prob);

Open file 'polyopt.task'
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 15              
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :       

We inspect the solution,

In [166]:
[prob.basis y]

15×2 Array{Main.Polyopt.Poly{Float64},2}:
 1.0      1.0
 z        -0.707154
 z^2      0.500067
 z^3      -0.353624
 z^4      0.250067
 x        -0.70706
 x*z      0.5
 x*z^2    -0.353577
 x*z^3    0.250033
 x^2      0.499933
 x^2*z    -0.35353
 x^2*z^2  0.25
 x^3      -0.353483
 x^3*z    0.249967
 x^4      0.249933

In [167]:
t - Polyopt.evalpoly(f, y[[6,2]])

-4.3935521887306095e-11

In [168]:
Polyopt.evalpoly(g, y[[6,2]]) >= 0

true

In [169]:
abs(Polyopt.evalpoly(h, y[[6,2]])) < 1e-5

true

## Duality interpretation

The returned solution consists objective value of the relaxation,

In [170]:
t

-1.4142135546838173

of two semidefinite matrices $X_1$ and $X_2$, corresponding to $f(x,z)$ and $g(x,z)$,

In [171]:
X

2-element Array{Array{Float64,2},1}:
 [0.7628216278898029 0.5495506795361622 … -0.13935554443399664 -0.28181668526824255; 0.5495506795361622 0.9498293954112726 … -0.0065747031915087545 -0.08103092108270234; … ; -0.13935554443399664 -0.0065747031915087545 … 0.8400241666007314 -0.0584076485131413; -0.28181668526824255 -0.08103092108270234 … -0.0584076485131413 0.5200868050328684]
 [0.3080595596991178 0.31304622346092614 0.1225507357258227; 0.31304622346092614 0.47521815406666035 -0.03247715865424768; 0.1225507357258227 -0.03247715865424768 0.2058487571368526]

a symmetric indefinite matrix $Z_1$ corresponding to $h(x,z$),

In [172]:
Z

1-element Array{Array{Float64,2},1}:
 [-0.6513919304615766 -0.1044790965819084 -0.20605475350457594; -0.1044790965819084 -0.5864426417472638 0.05840764851345477; -0.20605475350457594 0.05840764851345477 -0.5200868013212985]

and $y$ from which we extracted the coefficients for the optimal monomial solution,

In [173]:
y

15-element Array{Float64,1}:
  0.999999998416566
 -0.7071539245825795
  0.5000666716832701
 -0.3536241096485754
  0.2500666775889842
 -0.7070596300573022
  0.49999999174675425
 -0.3535769551016109
  0.2500333302119867
  0.4999333277019375
 -0.3535298146159062
  0.24999999497251063
 -0.3534826743529663
  0.24996666139817084
  0.24993333347494254

The dual interpretation expresses $f(x,z) - t = v_1^T X_1 v_1 + g(x,z) v_2^T X_2 v_2 + h(x,z) v_2^T Z_1 v_2$, in other words we express $f(x,z)-t$ as

$$f(x,z) - t = s_0(x,z) + g(x,z)s_1(x,z) - h(x,z) w(x, z)$$

where $s_0(x,z)$ and $s_1(x,z)$ are sums-of-squares, but $w(x,z)$ is not. Let us verify the expression,

In [174]:
v1 = Polyopt.monomials(2, [x,z])

6-element Array{Main.Polyopt.Poly{Int64},1}:
 1
 z
 z^2
 x
 x*z
 x^2

In [175]:
v2 = Polyopt.monomials(1, [x,z])

3-element Array{Main.Polyopt.Poly{Int64},1}:
 1
 z
 x

In [176]:
f-t

1.4142135546838173+z+x

In [177]:
using LinearAlgebra
round(truncate( dot(v1, X[1]*v1) + g*dot(v2, X[2]*v2) + h*dot(v2, Z[1]*v2), 1e-3 ), sigdigits=5)

1.4142+z+x